In [1]:
from simpletransformers.ner import NERModel
import pandas as pd
import logging
import re
from nltk.tokenize import TreebankWordTokenizer, sent_tokenize
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForTokenClassification
import openpyxl
from openpyxl import load_workbook
logging.basicConfig(level=logging.DEBUG)
transformers_logger = logging.getLogger('transformers')
transformers_logger.setLevel(logging.WARNING)

In [2]:
import os
filenames = os.listdir('/host/Sahithi/myenv/input/input')
connls = os.listdir('/host/Sahithi/myenv/output')

In [5]:
f1 = '/host/Sahithi/myenv/input/input/recent_00011.txt'
# for f1 in filenames:
with open(f1, "r") as cur:
    text=cur.read()
    #preprocessing of data
exceptions = {
        r'\d+mg': r'\g<0> |\g<1>',
        r'\d+:\d+': r'\g<0>',
        r'[\,\-\(\)]': r' \g<0> ',
        r'(?<!\d)\d+\.\d+(?!\d)': r'\g<0>',
        r'\d+/\d+': r'\g<0>',
        r'\d{1,2}[/-]\d{1,2}[/-]\d{2,4}': r'\g<0>',
        r'[A-Za-z]\d': r'\g<0>',
        r'[^\.\s]\.[^\.\s]': r'\g<0>',
        r'\d+\.\d+':r'\g<0>',
    }
tokenizer = TreebankWordTokenizer()
tokens = tokenizer.tokenize(text)
processed_tokens = []#list for 

a = ["COVID-19", "4/5Unable", "workstation:<ID", "patientÂ\x92s"]
sentences = [] # List to store processed sentences

for token in tokens:
    if token in a:  #making the list of processed tokens
        processed_tokens.append(token)
    elif token not in exceptions and (":" in token or "," in token or "/" in token or "." in token or "<" in token or ">" in token or "-" in token):
        if re.match(r'\d+/\d+', token) or re.match(r'\d+:\d+', token):
            processed_tokens.append(token)
        elif re.match(r'(?<!\d)\d+\.\d+(?!\d)', token):
            processed_tokens.append(token)
        else:
            split_tokens = re.split(r'(:|,|/|\.|<|>|-)', token)
            processed_tokens.extend([t for t in split_tokens if t])
    elif token in exceptions:
        processed_tokens.append(exceptions[token])
    else:
        processed_tokens.append(token)



    # Iterate over sentences and append processed tokens
current_sentence =""
sentences=""
for token in processed_tokens:
    current_sentence=current_sentence+" "+token

    if token.endswith('.') or token.endswith('!') or token.endswith('?'):
        sentences=sentences+" "+current_sentence
        current_sentence=""

#
# Append the remaining tokens as the last sentence
if current_sentence:
    sentences=sentences+current_sentence
c=[]
    #splitting the total sentences with maximum length as 350(it can be userdefined )
def process_input(s, max_length):
    if len(s) <= max_length:
        c.append(s)
    else:
        while len(s) > 0:
            chunk = s[:max_length] 
            c.append(chunk)
            s = s[max_length:]


max_length = 350
process_input(sentences, max_length)
    

In [6]:

# Annotated file for each document
a1 = 'annotator2_recent_00011_correct.conll'
a2="/host/Sahithi/myenv/output/"+a1
df = pd.read_csv(a2, delimiter='\n', header=None, quoting=3)
annotated_text = df[0].tolist()
a = []
for i in annotated_text:
    b = i.split(' ')
    a.append(b)


In [7]:
tokenizer = AutoTokenizer.from_pretrained("alvaroalon2/biobert_diseases_ner")
model = AutoModelForTokenClassification.from_pretrained("alvaroalon2/biobert_diseases_ner")
pipe = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")



DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /alvaroalon2/biobert_diseases_ner/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /alvaroalon2/biobert_diseases_ner/resolve/main/config.json HTTP/1.1" 200 0


In [8]:
# Making predictions here
for i in c: #c is the chunks of the data with max limit 350
    x=pipe(c)  


In [9]:
#predicted ones
p=[]
for i in x:
    p.extend(i)  #combining predictions dictioneries of the chunks into a list


In [10]:
p

[{'entity_group': '0',
  'score': 0.99999887,
  'word': 'Subjective : Follow - up Â Accompanied by her < OTHER >, < NAME _ PERSON >',
  'start': 2,
  'end': 75},
 {'entity_group': 'DISEASE',
  'score': 0.9998565,
  'word': 'MS',
  'start': 76,
  'end': 78},
 {'entity_group': '0',
  'score': 0.99996126,
  'word': 'Subtype : RRMS Â Last Relapse : none since the last visit Â Disease Modifying Therapy : Ocrevus ( last dose 2 / 2022 ) Prior Disease Modifying Therapy : Avonex ( 2002, on for one year, SE ), Betaseron ( 2003 - 1 / 2018 ) Interval history : progressively getting worse.',
  'start': 79,
  'end': 349},
 {'entity_group': '0',
  'score': 0.99999654,
  'word': 'The',
  'start': 1,
  'end': 4},
 {'entity_group': 'DISEASE',
  'score': 0.9999801,
  'word': 'l',
  'start': 5,
  'end': 6},
 {'entity_group': 'DISEASE',
  'score': 0.992101,
  'word': '##ymphedema',
  'start': 6,
  'end': 15},
 {'entity_group': '0',
  'score': 0.99999887,
  'word': 'is a problem. Able to walk short distance

In [11]:
predicted_tokens = []
for item in p:
    for word in item['word'].split():
        print(word)
        predicted_tokens.append([word,item['entity_group']])
    
    

Subjective
:
Follow
-
up
Â
Accompanied
by
her
<
OTHER
>,
<
NAME
_
PERSON
>
MS
Subtype
:
RRMS
Â
Last
Relapse
:
none
since
the
last
visit
Â
Disease
Modifying
Therapy
:
Ocrevus
(
last
dose
2
/
2022
)
Prior
Disease
Modifying
Therapy
:
Avonex
(
2002,
on
for
one
year,
SE
),
Betaseron
(
2003
-
1
/
2018
)
Interval
history
:
progressively
getting
worse.
The
l
##ymphedema
is
a
problem.
Able
to
walk
short
distance
to
bathroom
with
walker.
Energy
:
4
-
5
hours
at
night,
no
##cturia
X
1,
may
nap
during
the
day.
Cognitive
/
memory
:
some
word
finding
problems
but
otherwise
okay
Mood
:
okay
reduced
her
meds
Vision
:
Vc
glasses,
last
exam
January.
Some
days
worse
blurry
and
double
Bowel
:
urgency,
no
constipation
,
sometimes
make
it
and
sometimes
doesn
'
t.
Bladder
:
urgency,
in
##con
##tinence
Pain
:
legs
will
hurt
if
sits
for
a
long
time
in
a
regular
chair,
left
leg
is
hypersensitive,
burning
and
stabbing.
Balance
/
Gait
:
able
to
transfer
with
stand
and
pivot,
walker
to
get
to
bathroom
~
30
steps
t

In [12]:
len(predicted_tokens),len(processed_tokens),len(a)

(1508, 1527, 1540)

In [13]:
annotated_tokens = [[item[0],item[3]] for item in a]#making list of annotated_tokens and labels

In [14]:
len(predicted_tokens),len(annotated_tokens)

(1508, 1540)

In [16]:
tp = []
tn = []
fp = []
fn = []

for item in predicted_tokens:
    count = 0
    for items in annotated_tokens:
        if item[0] == items[0]: 
           
            count = 1
            if (item[1] == '0') & (items[1]=='O'):
                tn.append([item,items])
                break
            elif (item[1] == 'DISEASE') & (items[1]!='O'):
                tp.append([item,items])
                break
            elif (item[1] == 'DISEASE') & (items[1]=='O'):
                fp.append([item,items])
                break
            elif (item[1] == '0') & (items[1]!='O'):
                fn.append([item,items])
                break

    if count == 0:
        print(item,items)
        if item[1] == 'DISEASE':
            fp.append([[item,[item[0],'O']]])
        elif item[1] == '0':
            tn.append([[item,[item[0],'O']]])

          
        
         

                
        
         


['>,', '0'] ['CONTRAST', 'O']
['NAME', '0'] ['CONTRAST', 'O']
['_', '0'] ['CONTRAST', 'O']
['PERSON', '0'] ['CONTRAST', 'O']
['2', '0'] ['CONTRAST', 'O']
['2022', '0'] ['CONTRAST', 'O']
['2002,', '0'] ['CONTRAST', 'O']
['year,', '0'] ['CONTRAST', 'O']
['),', '0'] ['CONTRAST', 'O']
['worse.', '0'] ['CONTRAST', 'O']
['l', 'DISEASE'] ['CONTRAST', 'O']
['##ymphedema', 'DISEASE'] ['CONTRAST', 'O']
['problem.', '0'] ['CONTRAST', 'O']
['walker.', '0'] ['CONTRAST', 'O']
['night,', '0'] ['CONTRAST', 'O']
['##cturia', 'DISEASE'] ['CONTRAST', 'O']
['1,', '0'] ['CONTRAST', 'O']
['day.', '0'] ['CONTRAST', 'O']
['glasses,', '0'] ['CONTRAST', 'O']
['January.', '0'] ['CONTRAST', 'O']
['urgency,', '0'] ['CONTRAST', 'O']
['doesn', '0'] ['CONTRAST', 'O']
['t.', '0'] ['CONTRAST', 'O']
['urgency,', '0'] ['CONTRAST', 'O']
['##con', 'DISEASE'] ['CONTRAST', 'O']
['##tinence', 'DISEASE'] ['CONTRAST', 'O']
['chair,', '0'] ['CONTRAST', 'O']
['hypersensitive,', '0'] ['CONTRAST', 'O']
['stabbing.', '0'] ['CONTRAST

In [17]:
len(tn),len(tp),len(fp),len(fn), len(tn)+len(tp)+len(fp)+len(fn)

(1392, 2, 111, 3, 1508)

In [18]:
precision = len(tp)/(len(tp)+len(fp))
recall = len(tp)/(len(tp)+len(fn))
f1 = 2*(precision*recall)/(precision+recall)

In [19]:
precision,recall,f1


(0.017699115044247787, 0.4, 0.033898305084745756)

In [20]:
print(len(fn)+len(tn)+len(fp)+len(tp))

1508


In [29]:
total=[]
[total.extend(l) for l in (tp,tn,fp,fn)]
for i in total:
    print(i)

[['Multiple', 'DISEASE'], ['Multiple', 'B-RRMS']]
[['sclerosis', 'DISEASE'], ['sclerosis', 'I-RRMS']]
[['Subjective', '0'], ['Subjective', 'O']]
[[':', '0'], [':', 'O']]
[['Follow', '0'], ['Follow', 'O']]
[['-', '0'], ['-', 'O']]
[['up', '0'], ['up', 'O']]
[['Â', '0'], ['Â', 'O']]
[['Accompanied', '0'], ['Accompanied', 'O']]
[['by', '0'], ['by', 'O']]
[['her', '0'], ['her', 'O']]
[['<', '0'], ['<', 'O']]
[['OTHER', '0'], ['OTHER', 'O']]
[[['>,', '0'], ['>,', 'O']]]
[['<', '0'], ['<', 'O']]
[[['NAME', '0'], ['NAME', 'O']]]
[[['_', '0'], ['_', 'O']]]
[[['PERSON', '0'], ['PERSON', 'O']]]
[['>', '0'], ['>', 'O']]
[['Subtype', '0'], ['Subtype', 'O']]
[[':', '0'], [':', 'O']]
[['Â', '0'], ['Â', 'O']]
[['Last', '0'], ['Last', 'O']]
[['Relapse', '0'], ['Relapse', 'O']]
[[':', '0'], [':', 'O']]
[['none', '0'], ['none', 'O']]
[['since', '0'], ['since', 'O']]
[['the', '0'], ['the', 'O']]
[['last', '0'], ['last', 'O']]
[['visit', '0'], ['visit', 'O']]
[['Â', '0'], ['Â', 'O']]
[['Disease', '0'], ['

In [28]:

token=[]
predict=[]
annot=[]

for i in total:
    if(len(i)==2):
        token.append(i[0][0])
        predict.append(i[0][1])
        annot.append(i[1][1])
    elif(len(i)==1):
        for j in i:
            token.append(j[0][0])
            predict.append(j[0][1])
            annot.append(j[1][1])
#making a dataframe with three columns 'token','predictiions' and 'annotated'
output=pd.DataFrame({
                'token':token,
                'predictions':predict,
                'annotated':annot,
            })
print(output)



print(len(token),len(annot))

           token predictions annotated
0       Multiple     DISEASE    B-RRMS
1      sclerosis     DISEASE    I-RRMS
2     Subjective           0         O
3              :           0         O
4         Follow           0         O
...          ...         ...       ...
1503   ##patitis     DISEASE         O
1504           B     DISEASE         O
1505        RRMS           0    B-RRMS
1506        RRMS           0    B-RRMS
1507        EDSS           0    B-EDSS

[1508 rows x 3 columns]
1508 1508
